In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\上市信息财务信息-利润表.xlsx')

In [ ]:
data

In [ ]:
#将单位全部统一为以元为单位
def convert(cell):
    pattern1=(r'(-?\d+\.?\d+)亿')
    pattern2=(r'(-?\d+\.?\d+)万亿')
    pattern3=(r'(-?\d+\.?\d+)万')
    pattern4=(r'(-?\d+\.?\d+)')
    if re.findall(pattern1,cell)!=[]:
        cell=float(re.findall(pattern1, cell)[0])/10000
    elif re.findall(pattern2,cell)!=[]:
        cell=float(re.findall(pattern2, cell)[0])
    elif re.findall(pattern3,cell)!=[]:
        cell=float(re.findall(pattern3, cell)[0])/10000000
    elif re.findall(pattern4,cell)!=[]:
        cell=float(re.findall(pattern4, cell)[0])/100000000000
    elif cell=='nan'or cell=='--':
        cell=np.nan
    else: cell=float(cell)
    return cell

In [ ]:
data.iloc[:,3:]=data.iloc[:,3:].applymap(str).applymap(convert)

In [ ]:
data

In [ ]:
#查看出现较多的时间
data['日期'].value_counts()

In [ ]:
#补全时间，规整时间的格式
pattern='\d\d\d\d-\d\d-\d\d'
for i in range(len(data)):
    if re.findall(pattern,data.iloc[i,2])==[]:
        data.iloc[i,2]='20'+data.iloc[i,2]

In [ ]:
#去除企业编号与日期值同时相同的行，保留第一次出现的行
data.drop_duplicates(subset=['企业编号','日期'],keep='first',inplace=True)

In [ ]:
#再次查看出现较多的时间
data['日期'].value_counts()

In [ ]:
#将数量较少的日期全部丢弃
timelist=list(['2017-09-30','2017-12-31','2018-06-30','2018-09-30','2017-06-30','2018-03-31'])
data=data[data['日期'].isin(timelist)]
data

In [ ]:
data.sort_values(by=['企业编号','日期'],inplace=True)

In [ ]:
data

In [ ]:
#查看排序结果是否达到目标效果
data.columns=pd.Index(['企业编号', '标题', '日期', '$营业收入(元)', '$营业成本(元)', '$销售费用(元)', '$财务费用(元)',
       '$管理费用(元)', '$资产减值损失(元)', '$投资收益(元)', '$营业利润(元)', '$利润总额(元)', '$所得税(元)',
       '$归属母公司所有者净利润(元)'])

In [ ]:
data

In [ ]:
#'营业收入(元)',营业成本(元),
# data['营业成本(元)'].describe()
# data['营业成本(元)'].sort_values(ascending=False)

In [ ]:
data=data.set_index(['企业编号','日期'])
data=data.unstack().reset_index().set_index('企业编号').T.drop('标题')
data

In [ ]:
data=data.reindex(columns=list(range(1001,4001)))
data.to_pickle('上市信息财务信息-利润表.pickle')
data.index